# ライブラリのインポート

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
from datasets import load_dataset
from datasets import load_from_disk
import faiss

/home/st6322011/anaconda3/envs/NLP/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-03 15:03:32.879757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 15:03:35.197406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# モデルのダウンロード

In [4]:
device = "cpu"
model = SentenceTransformer("cl-nagoya/sup-simcse-ja-base", device=device)

/home/st6322011/anaconda3/envs/NLP/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/st6322011/anaconda3/envs/NLP/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [55]:
sentences = ["こんにちは、世界！", "文埋め込み最高！文埋め込み最高と叫びなさい", "極度乾燥しなさい"]
results = model.encode(sentences)
print(results)

[[ 0.90896523 -0.10347278  0.49861845 ... -0.07458906  0.8871121
  -0.5814303 ]
 [ 1.1992244  -0.31974453 -0.3879857  ... -0.63752735  0.1330044
  -0.17121182]
 [ 0.30453068  0.42420724 -0.50362414 ... -0.5128267  -0.11586949
  -0.74181527]]


# データセットのダウンロードと前処理

In [56]:
wiki = load_dataset("graelo/wikipedia", "20230901.ja")

/home/st6322011/anaconda3/envs/NLP/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for graelo/wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/graelo/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [57]:
wiki

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 1383531
    })
})

In [58]:
wiki['train']

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1383531
})

In [59]:
# print(wiki['train'][0]['text'])

In [60]:
wiki = wiki.map(lambda row: {'text': row['text'].split('。')[0]})

In [61]:
for i in range(10):
    print("-"*100)
    print(wiki['train'][i]['text'])

----------------------------------------------------------------------------------------------------
アンパサンド（&, ）は、並立助詞「…と…」を意味する記号である
----------------------------------------------------------------------------------------------------
生物学（せいぶつがく、、）とは、生命現象を研究する、自然科学の一分野である
----------------------------------------------------------------------------------------------------
ゴーダチーズ（ , 、 ）は、オランダを代表するチーズ
----------------------------------------------------------------------------------------------------
ブラックミュージック () あるいは黒人音楽（こくじんおんがく）とは、アメリカの黒人発祥の音楽の総称を表す言葉
----------------------------------------------------------------------------------------------------
著作権（ちょさくけん、、コピーライト）は、作品を創作した者が有する権利である
----------------------------------------------------------------------------------------------------
『うる星やつら』（うるせいやつら、ラテン文字表記: Urusei Yatsura）は、高橋留美子による日本の漫画作品
----------------------------------------------------------------------------------------------------
高橋 しん（たかはし しん、本名：高橋 真（たかはし しん）、男性、1967年（

In [62]:
# データセットの一部を抽出
#wiki_train = wiki['train']
wiki_train = wiki['train'].select(range(100))

# 文の埋め込みを計算して保存

In [63]:
# 段落データのすべての事例に埋め込みを付与
wiki_train = wiki_train.map(
    lambda data: {"embeddings": model.encode(data['text']) },
    batch_size=256,
    batched=True,
)  


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [64]:
wiki_train.save_to_disk("embedded")

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

# Faissの利用

In [2]:
wiki_train = load_from_disk("save/embedded_paragraphs")

In [5]:
embedding_dim = model.get_sentence_embedding_dimension()
index = faiss.IndexFlatIP(embedding_dim)
wiki_train.add_faiss_index("embeddings", custom_index=index)

  0%|          | 0/1384 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'url', 'title', 'text', 'embeddings'],
    num_rows: 1383531
})

In [12]:
query = "スマホを新しいものに変えようと思っているんだ"

scores, retrieved_examples = wiki_train.get_nearest_examples(
    "embeddings", model.encode(query), k=20
)
print(f'type : {type(scores)}')
print(f'type : {type(retrieved_examples)}')
print(f'retrieved_examples.keys() : {retrieved_examples.keys()}')

type : <class 'numpy.ndarray'>
type : <class 'dict'>
retrieved_examples.keys() : dict_keys(['id', 'url', 'title', 'text', 'embeddings'])


In [13]:
texts = retrieved_examples["text"]
print("score    | text")
print("-"*200)
for score, text in zip(scores, texts):
    print(f'{score:.3f}  | {text}')

score    | text
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
258.020  | GreenphoneとはTrolltechがGUIとほとんどフリーやオープンソースソフトウェアを使っているLinuxを組み込んだアプリケーションプラットフォームであるQtopia Phone Editionで開発したスマートフォンである
256.336  | スマホ同期 (英: Your Phone)は、AndroidやiOSのデバイスをWindows 10デバイスに接続するために、Microsoft Windows 10用に開発されたアプリ
254.406  | iPhone（アイフォーン）は、Appleが設計・販売しているスマートフォン
251.605  | 『異世界はスマートフォンとともに
251.080  | Openmoko LinuxとはOpenmokoプロジェクトにより開発が進められているスマートフォン向けのOSである
248.402  | Google カメラとは、Googleによって開発された、モバイル端末のGoogle Pixel向けのカメラアプリ
248.029  | ガラホとは、スマートフォン用のOSや半導体部品を転用して開発された、日本国内向けフィーチャー・フォン（いわゆるガラパゴスケータイ）の一種を指す新造語
247.353  | iPhone 11（アイフォーン イレブン）は、Appleのスマートフォン
246.656  | BlackBerry Z10とはBlackBerry（旧名リサーチ・イン・モーション）が開発したタッチパネル型スマートフォンである
246.646  | スマートフォン（）は、パーソナルコンピュータなみの機能をもたせた携帯電話やPHSの総称
246.552  | iPhone（アイフォーン）は、Appleが販売したスマートフォンである
246.410  | iPhon